# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

In [36]:
import pandas as pd, re

r = pd.read_csv("recipes.tsv", sep="\t")[["recipe_slug","recipe_title"]]
t = pd.read_csv("recipe-tags.tsv", sep="\t")

r["rating_raw"] = 2  # default = 2 → 0.25 (less neutral)
H = r"(?i)\b(?:vegan|vegetarian|salad|grilled|baked|steamed|quinoa|kale|spinach|broccoli|chickpea|lentil|tofu)\b"
B = r"\bbacon\b"

# Healthy → 5; Bacon (title or tags) → 1
r.loc[r.recipe_title.str.contains(H, regex=True, na=False), "rating_raw"] = 5
r.loc[r.recipe_title.str.contains(B, case=False, na=False), "rating_raw"] = 1
bacon_slugs = set(t.loc[t.recipe_tag.str.contains(B, case=False, na=False), "recipe_slug"])
r.loc[r.recipe_slug.isin(bacon_slugs), "rating_raw"] = 1

# Ensure ≥10 zeros and ≥10 ones after mapping
need0 = max(0, 10 - (r["rating_raw"]==1).sum())
if need0: r.loc[r[r["rating_raw"]>1].sample(need0, random_state=0).index, "rating_raw"] = 1
need1 = max(0, 10 - (r["rating_raw"]==5).sum())
if need1: r.loc[r[r["rating_raw"]<5].sample(need1, random_state=0).index, "rating_raw"] = 5

r["rating"] = (r["rating_raw"] - 1) * 0.25
r[["recipe_slug","rating"]].to_csv("ratings.tsv", sep="\t", index=False)
print("ratings.tsv saved (bacon=0, healthy=1, default=0.25)")


ratings.tsv saved (bacon=0, healthy=1, default=0.25)


Submit "ratings.tsv" in Gradescope.

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [37]:
import pandas as pd

ratings = pd.read_csv("ratings.tsv", sep="\t")
tags    = pd.read_csv("recipe-tags.tsv", sep="\t")

ratings["slug_norm"] = ratings.recipe_slug.str.strip().str.lower()
tags["slug_norm"]    = tags.recipe_slug.str.strip().str.lower()

X = pd.crosstab(tags["slug_norm"], tags["recipe_tag"]).clip(0,1)
X = X.reindex(ratings["slug_norm"]).fillna(0).astype(int)

features = pd.concat(
    [ratings["recipe_slug"].reset_index(drop=True),
     pd.Series(1, index=X.index, name="bias").reset_index(drop=True),
     X.reset_index(drop=True)], axis=1)

features.to_csv("features.tsv", sep="\t", index=False)
print("features.tsv saved")


features.tsv saved


Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [44]:
import pandas as pd
from sklearn.linear_model import Ridge

F = pd.read_csv("features.tsv", sep="\t")
y = pd.read_csv("ratings.tsv",  sep="\t")["rating"].values
X = F.drop(columns=["recipe_slug"])

m = Ridge(alpha=1.0, fit_intercept=False).fit(X, y)
pd.Series(m.coef_, index=X.columns).drop("bias") \
  .rename_axis("recipe_tag").reset_index(name="coefficient") \
  .to_csv("model.tsv", sep="\t", index=False)
print("model.tsv saved")


model.tsv saved


Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not include the bias.

Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [45]:
import pandas as pd
from sklearn.linear_model import Ridge

F = pd.read_csv("features.tsv", sep="\t")
y = pd.read_csv("ratings.tsv",  sep="\t")["rating"].values
X = F.drop(columns=["recipe_slug"])

m = Ridge(alpha=1.0, fit_intercept=False).fit(X, y)
pd.DataFrame({"recipe_slug": F["recipe_slug"],
              "score_estimate": m.predict(X).clip(0,1)}) \
  .to_csv("estimates.tsv", sep="\t", index=False)
print("estimates.tsv saved")


estimates.tsv saved


Submit "estimates.tsv" in Gradescope.

In [48]:
import pandas as pd
e = pd.read_csv("estimates.tsv", sep="\t")
print(e.sort_values("score_estimate", ascending=False).head(10))
print(e.sort_values("score_estimate").head(10))


                               recipe_slug  score_estimate
99             spinach-and-ricotta-lasagna        0.956688
72                              taco-salad        0.948088
83                 spinach-and-feta-quiche        0.933841
53                             nacho-salad        0.933067
33                                   tacos        0.902884
85  soba-noodle-salad-with-peanut-dressing        0.895582
2                         bacon-fried-rice        0.855783
78                          spinach-quiche        0.834225
98             vegetarian-mushroom-lasagna        0.806635
42      spinach-and-ricotta-stuffed-shells        0.761141
                         recipe_slug  score_estimate
11            bacon-wrapped-scallops        0.000000
80                   quiche-lorraine        0.000585
92      bacon-wrapped-shrimp-skewers        0.002050
41              maple-bacon-pancakes        0.003153
93             bacon-wrapped-chicken        0.013204
6       bacon-chocolate-chip-cook

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [46]:
import pandas as pd, numpy as np

F = pd.read_csv("features.tsv", sep="\t")
R = pd.read_csv("ratings.tsv",  sep="\t")
X = F.drop(columns=["recipe_slug"]).to_numpy(float); r = R["rating"].to_numpy(float)
lam, alpha = 1.0, 2.0
A = lam*np.eye(X.shape[1]) + X.T@X
b = X.T@r
Ainv = np.linalg.inv(A); theta = Ainv@b
ub = (X@theta) + alpha*np.sqrt(np.einsum("ij,jk,ik->i", X, Ainv, X))
pd.DataFrame({"recipe_slug": F["recipe_slug"], "score_bound": ub}).to_csv("bounds.tsv", sep="\t", index=False)
print("bounds.tsv saved")


bounds.tsv saved


Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [47]:
import pandas as pd, numpy as np
F = pd.read_csv("features.tsv", sep="\t")
R = pd.read_csv("ratings.tsv",  sep="\t").set_index("recipe_slug")["rating"]
X = F.drop(columns=["recipe_slug"]).to_numpy(float); slugs = F["recipe_slug"].tolist()
lam, alpha = 1.0, 2.0
A = lam*np.eye(X.shape[1]); b = np.zeros(X.shape[1]); recs=[]
for _ in range(100):
    Ainv = np.linalg.inv(A); theta = Ainv@b
    ucb = (X@theta) + alpha*np.sqrt(np.einsum("ij,jk,ik->i", X, Ainv, X))
    i = int(np.argmax(ucb)); s = slugs[i]; r = float(R.loc[s])
    recs.append((s, float(ucb[i]), r))
    x = X[i]; A += np.outer(x,x); b += r*x
pd.DataFrame(recs, columns=["recipe_slug","score_bound","reward"]).to_csv("recommendations.tsv", sep="\t", index=False)
print("recommendations.tsv saved")


recommendations.tsv saved


Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.

In [35]:
import pandas as pd, numpy as np

P="."
R = pd.read_csv(f"{P}/ratings.tsv", sep="\t")
F = pd.read_csv(f"{P}/features.tsv", sep="\t")
M = pd.read_csv(f"{P}/model.tsv", sep="\t")
E = pd.read_csv(f"{P}/estimates.tsv", sep="\t")
B = pd.read_csv(f"{P}/bounds.tsv", sep="\t")
Q = pd.read_csv(f"{P}/recommendations.tsv", sep="\t")

def ok(name,cond,msg=""):
    print(f"[{'OK' if cond else 'FAIL'}] {name}", ("" if cond else f"→ {msg}"))

# Part 1
ok("Part1 ratings cols", list(R.columns)==["recipe_slug","rating"], "cols must be [recipe_slug,rating]")
ok("Part1 extremes", (R.rating.eq(0).sum()>=10 and R.rating.eq(1).sum()>=10),
   f"zeros={R.rating.eq(0).sum()}, ones={R.rating.eq(1).sum()}")

# Part 2
ok("Part2 order match", (F.recipe_slug.tolist()==R.recipe_slug.tolist()), "features order must match ratings")
ok("Part2 bias col", "bias" in F.columns, "add bias=1")
X = F.drop(columns=["recipe_slug","bias"])
ok("Part2 one-hot 0/1", (X.max().le(1).all() and X.min().ge(0).all()), "tag cols must be 0/1")

# Part 3
ok("Part3 model cols", list(M.columns)==["recipe_tag","coefficient"], "cols must be [recipe_tag,coefficient]")

# Part 4
ok("Part4 estimates cols", list(E.columns)==["recipe_slug","score_estimate"], "cols must be [recipe_slug,score_estimate]")
ok("Part4 estimate range", (E.score_estimate.between(0,1).all()), "clip to [0,1]")

# Part 5
ok("Part5 bounds cols", list(B.columns)==["recipe_slug","score_bound"], "cols must be [recipe_slug,score_bound]")
ok("Part5 one-per-recipe", (len(B)==len(R)), "bounds must have one row per recipe")

# Part 6
ok("Part6 recs cols", list(Q.columns)==["recipe_slug","score_bound","reward"],
   "cols must be [recipe_slug,score_bound,reward]")
ok("Part6 100 rounds", (len(Q)==100), f"have {len(Q)} rows")
ok("Part6 reward range", Q.reward.between(0,1).all(), "rewards must be from ratings.tsv")

# Optional bacon sanity
try:
    print("bacon-fried-rice estimate:",
          float(E.set_index("recipe_slug").loc["bacon-fried-rice"]))
except Exception:
    pass


[OK] Part1 ratings cols 
[OK] Part1 extremes 
[OK] Part2 order match 
[OK] Part2 bias col 
[OK] Part2 one-hot 0/1 
[OK] Part3 model cols 
[OK] Part4 estimates cols 
[OK] Part4 estimate range 
[OK] Part5 bounds cols 
[OK] Part5 one-per-recipe 
[OK] Part6 recs cols 
[OK] Part6 100 rounds 
[OK] Part6 reward range 
bacon-fried-rice estimate: 0.8676589574642223


/tmp/ipykernel_2330/1976842649.py:45: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(E.set_index("recipe_slug").loc["bacon-fried-rice"]))
